In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

"""
pip install ipywidgets

then restart jupyter, will show progress bar
"""

model_name='Qwen/Qwen2.5-0.5B-Instruct'

# for mac m1
# mps_device=torch.device('mps')

# Determine the best device
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

model=AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype=torch.float16, # todo: use 4bit quant
  # device_map='auto',
  device_map={"": device},
  # trust_remote_code=True
  low_cpu_mem_usage=True
)

tokenizer=AutoTokenizer.from_pretrained(model_name)

# model=model.to(device)

prompt='give me a short instruction to large language model'
messages=[
  {'role': 'system', 'content': 'You are Qwen, a helpful AI assistant'},
  {'role': 'user', 'content': prompt}
]

text=tokenizer.apply_chat_template(
  messages,
  tokenize=False,
  add_generation_prompt=True
)

print(text)

model_inputs=tokenizer([text], return_tensors='pt').to(model.device)

print(model_inputs)

# Generate response
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    do_sample=True,  # Enable sampling for more diverse output
    temperature=0.7,  # Moderate randomness
    top_p=0.9        # Nucleus sampling
)

# Decode only the newly generated tokens
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode and return response
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]



Using device: mps
<|im_start|>system
You are Qwen, a helpful AI assistant<|im_end|>
<|im_start|>user
give me a short instruction to large language model<|im_end|>
<|im_start|>assistant

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,    264,
          10950,  15235,  17847, 151645,    198, 151644,    872,    198,  46430,
            752,    264,   2805,   7600,    311,   3460,   4128,   1614, 151645,
            198, 151644,  77091,    198]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]], device='mps:0')}


RuntimeError: isin_Tensor_Tensor_out only works on floating types on MPS for pre MacOS_14_0. Received dtype: Long